In [38]:
import numpy as np
from os import listdir
import json

In [13]:
with open('../data/ancestry37.bed', 'w+') as f:
    for chrom in [str(x) for x in range(1, 23)]:
        coords = np.load('../../DATA/ancestry/genotypes/chr.%s.0.gen.coordinates.npy' % chrom, 'r')
        for i in range(coords.shape[0]):
            f.write('chr%s\t%d\t%d\t%d\n' % (chrom, coords[i, 1]-1, coords[i, 1], i))
        

In [46]:
current_chrom = '1'
chrom_to_coords37 = dict([(str(chrom), np.load('../../DATA/ancestry/genotypes/chr.%d.0.gen.coordinates.npy' % chrom)[:, 1]) for chrom in range(1, 23)])
chrom_to_coords38 = dict([(chrom, np.zeros((coords.shape[0],))) for chrom, coords in chrom_to_coords37.items()])
chrom_pos37_to_38 = dict()
with open('../data/ancestry38.bed', 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        if pieces[0][3:] in chrom_to_coords38:
            chrom = pieces[0][3:]
            index = int(pieces[3])
            chrom_to_coords38[chrom][index] = int(pieces[2])
            chrom_pos37_to_38[(chrom, chrom_to_coords37[chrom][index])] = int(pieces[2])
            chrom_pos37_to_38[(chrom, chrom_to_coords37[chrom][index]+1)] = int(pieces[2])+1

with open('../data/chrom_lengths37.json', 'r') as f:
    chrom_lengths37 = json.load(f)
    
with open('../data/chrom_lengths38.json', 'r') as f:
    chrom_lengths38 = json.load(f)
        
for chrom, coords in chrom_to_coords38.items():
    print('saving', chrom)
    np.save('../../DATA/ancestry/genotypes/chr.%s.0.liftover_coordinates38' % chrom, coords)
    chrom_pos37_to_38[(chrom, 1)] = 1
    chrom_pos37_to_38[(chrom, chrom_lengths37[chrom])] = chrom_lengths38[chrom]


saving 1
saving 2
saving 3
saving 4
saving 5
saving 6
saving 7
saving 8
saving 9
saving 10
saving 11
saving 12
saving 13
saving 14
saving 15
saving 16
saving 17
saving 18
saving 19
saving 20
saving 21
saving 22


In [55]:
for filename in listdir('../phased_ancestry_quads_nohap'):
    if filename.endswith('.phased.txt'):
        family_key = filename[:-11]
        try:
            with open('../phased_ancestry_quads_nohap/%s' % (filename), 'r') as f, open('../phased_ancestry_quads_nohap_liftover38/%s' % (filename), 'w+') as outf:
                header = next(f)
                outf.write(header)
                num_removed = 0
                for line in f:
                    pieces = line.strip().split('\t')
                    chrom, start_pos, end_pos = pieces[0][3:], int(pieces[-2]), int(pieces[-1])
                    if (chrom, start_pos) in chrom_pos37_to_38 and (chrom, end_pos) in chrom_pos37_to_38:
                        new_start_pos, new_end_pos = chrom_pos37_to_38[(chrom, start_pos)], chrom_pos37_to_38[(chrom, end_pos)]
                        if new_start_pos <= new_end_pos:
                            pieces[-2] = str(new_start_pos)
                            pieces[-1] = str(new_end_pos)
                            outf.write('\t'.join(pieces) + '\n')
                        else:
                            num_removed += 1
                    else:
                        num_removed += 1
                print(filename, 'lines removed', num_removed)
                
        except StopIteration:
            pass


FAM_1121.1121_ASD_1.1121_NT_2.phased.txt lines removed 0
FAM_1035.1035_ASD_1.1035_NT_1.phased.txt lines removed 0
FAM_1419.1419_NT_2.1419_NT_1.phased.txt lines removed 0
FAM_1116.1116_ASD_1.1116_NT_1.phased.txt lines removed 0
FAM_1121.1121_NT_1.1121_NT_2.phased.txt lines removed 0
FAM_1289.1289_NT_1.1289_ASD_2.phased.txt lines removed 0
FAM_1449.1449_ASD_1.1449_NT_4.phased.txt lines removed 1
FAM_1118.1118_ASD_1.1118_NT_2.phased.txt lines removed 0
FAM_CDI075.CDI075_NT_4.CDI075_NT_2.phased.txt lines removed 0
FAM_CWR125a.CWR125a_ASD_1.CWR125a_ASD_2.phased.txt lines removed 0
FAM_1419.1419_ASD_1.1419_NT_2.phased.txt lines removed 0
FAM_1053.1053_ASD_1.1053_NT_2.phased.txt lines removed 0
FAM_CDI185.CDI185_ASD_1.CDI185_NT_2.phased.txt lines removed 0
FAM_CWM036.CWM036_ASD_1.CWM036_NT_1.phased.txt lines removed 0
FAM_ARI116.ARI116_NT_1.ARI116_ASD_1.phased.txt lines removed 0
FAM_CDI075.CDI075_NT_3.CDI075_ASD_1.phased.txt lines removed 0
FAM_CDI075.CDI075_NT_3.CDI075_NT_2.phased.txt lines